In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from torch.autograd import Variable
from random import shuffle
from scipy.stats import beta
import scipy.special
import sys 
sys.path.append('../')
from utils import *

%matplotlib inline

import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.figsize'] = (15, 5)

In [ ]:
class MDN(nn.Module):
    def __init__(self, ndim_input=2, ndim_output=2, n_hidden=10, n_components=1):
        super(MDN, self).__init__()
        
        self.ndims = ndim_output 
        self.n_components = n_components
        # the number of entries in the upper triangular Choleski transform matrix of the precision matrix 
        self.utriu_entries = int(self.ndims * (self.ndims - 1) / 2) + self.ndims

        # input layer 
        self.fc_in = nn.Linear(ndim_input, n_hidden)
        # activation 
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

        # output layer the mean estimates 
        self.mu_out = nn.Linear(n_hidden, ndim_output * n_components)

        # output layer to precision estimates 
        # the upper triangular matrix for D-dim Gaussian has m = (D**2 + D) / 2 entries 
        # this should be a m-vector for every component. currently it is just a scalar for every component. 
        # or it could be a long vector of length m * k, i.e, all the k vector stacked. 
        self.U_out = nn.Linear(n_hidden, self.utriu_entries * n_components)
        
        # additionally we have the mixture weights alpha 
        self.alpha_out = nn.Linear(n_hidden, n_components)

    def forward(self, x):
        batch_size = x.size()[0]
        
        out = self.fc_in(x)
        act = self.tanh(out)
        
        out_mu = self.mu_out(act)
        out_mu = out_mu.view(batch_size, self.ndims, self.n_components)
        out_alpha = self.softmax(self.alpha_out(act))

        # get activate of upper triangle U vector
        U_vec = self.U_out(act)
        # prelocate U matrix 
        U_mat = Variable(torch.zeros(batch_size, self.n_components, self.ndims, self.ndims))
        
        # assign vector to upper triangle of U 
        (idx1, idx2) = np.triu_indices(self.ndims)
        U_mat[:, :, idx1, idx2] = U_vec
        # apply exponential to get positive diagonal
        (idx1, idx2) = np.diag_indices(self.ndims)
        U_mat[:, :, idx1, idx2] = torch.exp(U_mat[:, :, idx1, idx2])
        
        return (out_mu, U_mat, out_alpha)

In [ ]:
def multivariateMoG_loss(y, mu, U, alpha):
    
    result = multivariateMoG_pdf(y, mu, U, alpha, log=True)    
    
    result = torch.mean(result)  # mean over batch
    return -result

def multivariateMoG_pdf(y, mus, Us, alphas, log=False): 
    
    # get params: batch size N, ndims D, ncomponents K
    N, D, K = mus.size()
        
    # prelocate matrix for log probs of each Gaussian component
    log_probs_mat = Variable(torch.zeros(N, K))
    
    # take weighted sum over components to get log probs 
    for k in range(K): 
        log_probs_mat[:, k] = multivariate_normal_pdf(X=y, mus=mus[:, :, k], Us=Us[:, k, :, :], log=True)
    
    # now apply the log sum exp trick: sum_k alpha_k * N(Y|mu, sigma) = sum_k exp(log(alpha_k) + log(N(Y| mu, sigma)))
    # this give the log MoG density over the batch
    log_probs_batch = my_log_sum_exp(torch.log(alphas) + log_probs_mat, axis=1)  # sum over component axis=1
    
    # return log or linear density dependent on flag: 
    if log: 
        result = log_probs_batch
    else: 
        result = torch.exp(log_probs_batch)
    
    return result

In [ ]:
def generate_2DGaussian_dataset(n_samples, sample_size, prior, data_cov=None):

    X = []
    thetas = []
    
    if data_cov is None: 
        data_cov = .5 * np.eye(2)
    
    for i in range(n_samples): 
        # sample from the prior 
        theta = prior.rvs()

        # generate samples with mean from prior and unit variance 
        x = scipy.stats.multivariate_normal.rvs(mean=theta, cov=data_cov, size=sample_size).reshape(sample_size, 2)
        
        sx = stats_ND_Gaussian(x)

        # as data we append the summary stats
        X.append(sx) 
        thetas.append([theta])   
    
    return np.array(X).squeeze(), np.array(thetas).squeeze()

def stats_ND_Gaussian(x): 
    """
    Calculate the sufficient statistics of a multivariate Gaussian sample x
    """
    return np.array([np.sum(x, axis=0).astype(float)])
        
def train(X, Y, n_epochs=500, n_minibatch=50):
    dataset_train = [(x, y) for x, y in zip(X, Y)]
    
    losses = []
    
    for epoch in range(n_epochs): 
        bgen = batch_generator(dataset_train, n_minibatch)

        for j, (x_batch, y_batch) in enumerate(bgen):
            x_var = Variable(torch.Tensor(x_batch))
            y_var = Variable(torch.Tensor(y_batch))
                        
            (out_mu, out_U, out_alpha) = model(x_var)
            loss = multivariateMoG_loss(y_var, out_mu, out_U, out_alpha)
            
            optim.zero_grad()
            loss.backward()
            optim.step()
            
            losses.append(loss.data.numpy())

        if (epoch + 1) % 10 == 0:
            print("[epoch %04d] loss: %.4f" % (epoch + 1, loss.data[0]))
            
    return losses

In [ ]:
model = MDN(ndim_input=2, n_components=3)
optim = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
n_samples = 10000
sample_size = 10
    
# prior on the mean
data_cov = 0.5 * np.eye(2)
prior = scipy.stats.multivariate_normal(mean=[0., 0.], cov=2. * np.eye(2))

X, Y = generate_2DGaussian_dataset(n_samples, sample_size, prior, data_cov)
X, norm = normalize(X)

In [ ]:
prior.mean.size

In [ ]:
loss = train(X, Y, n_epochs=50, n_minibatch=100)
plt.plot(loss);

In [ ]:
# observe data 
true_mu = [-.5, .1]
true_cov = data_cov
xo = scipy.stats.multivariate_normal.rvs(mean=true_mu, cov=true_cov, size=sample_size).reshape(sample_size, 2)
# gets stats 
statso = stats_ND_Gaussian(xo)
# normalize 
statso, norm = normalize(statso, norm)
# cast to torch 
statso = Variable(torch.Tensor(statso))

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(211)
plt.hist2d(X[:, 0], X[:, 1])
plt.title('Samples generated by the model: 2D Gaussian with prior on $\mu$');
plt.colorbar();
plt.subplot(212)
plt.title('observed data')
plt.hist2d(xo[:, 0], xo[:, 1])
plt.colorbar();

In [ ]:
# predict with observed data 
(out_mu, out_U, out_alpha) = model(statso.view(1, 2))

In [ ]:
post_mean, post_cov = calculate_multivariate_normal_mu_posterior(xo, data_cov, sample_size, 
                                                                 prior.mean, prior.cov)
postana = scipy.stats.multivariate_normal(mean=post_mean, cov=post_cov)

In [ ]:
r = 1.
x, y = np.mgrid[-r:r:.01, -r:r:.01]
pos = np.dstack((x, y))

In [ ]:
pos.shape[:-1]

In [ ]:
# calculate the grid of PDF values for the MoG: 
# extract torch variables for every component and take weighted sum with alpha weights
posterior_grid = np.zeros((200, 200))

# for every component 
for k in range(3): 
    alpha = out_alpha[0, k].data.numpy()
    mean = out_mu[0, :, k].data.numpy()
    U = out_U[0, k, ].data.numpy()
    # get cov from Choleski transform 
    cov = np.linalg.inv(U.T.dot(U))
    # add to result, weighted with alpha 
    posterior_grid += alpha * scipy.stats.multivariate_normal.pdf(x=pos, mean=mean, cov=cov)

In [ ]:
plt.figure(figsize=(15, 12))
plt.subplot(311)
plt.contourf(x, y, prior.pdf(pos))
plt.plot(true_mu[0], true_mu[1], 'ro')
plt.title('Prior')

plt.subplot(312)
plt.contourf(x, y, posterior_grid)
plt.plot(true_mu[0], true_mu[1], 'ro')
plt.title('Posterior')

plt.subplot(313)
plt.contourf(x, y, postana.pdf(pos))
plt.title('Analytical Posterior')
plt.plot(true_mu[0], true_mu[1], 'ro');